In [1]:
import pandas as pd

plogs = pd.read_csv('plogs.csv')
pdets = pd.read_csv('pdets.csv')

print("---- plogs.csv ----")
print(plogs.shape)
print(plogs.head())
print(plogs.nunique())
print(plogs.isnull().sum())

print("\n---- pdets.csv ----")
print(pdets.shape)
print(pdets.head())
print(pdets.nunique())
print(pdets.isnull().sum())

print("\nNumerical Description (plogs)")
print(plogs.describe())

print("\nNumerical Description (pdets)")
print(pdets.describe())

categorical_plogs = plogs.select_dtypes(include=['object', 'category']).columns
print("\nCategorical Description (plogs)")
print(plogs[categorical_plogs].describe())

categorical_pdets = pdets.select_dtypes(include=['object', 'category']).columns
print("\nCategorical Description (pdets)")
print(pdets[categorical_pdets].describe())

IMP_PLOGS_FIELDS = ['problem_id', 'student_id', 'assignment_id', 'problem_completed', 'correct']
IMP_PDETS_FIELDS = ['problem_id', 'problem_type', 'content_source', 'skills']

print("\n-- plogs important field value counts --")
for col in IMP_PLOGS_FIELDS:
    print(f"Value counts for {col}:")
    print(plogs[col].value_counts(dropna=False).head(10))  # top 10

print("\n-- pdets important field value counts --")
for col in IMP_PDETS_FIELDS:
    print(f"Value counts for {col}:")
    print(pdets[col].value_counts(dropna=False).head(10))  # top 10

merged = pd.merge(
    plogs,
    pdets,
    how='left',
    on='problem_id'
)
print("\nMerged shape:", merged.shape)
print(merged.head())

missing_metadata_count = merged['content_source'].isnull().sum()
print(f"Logs with no matching problem metadata: {missing_metadata_count}")

merged.to_csv('plogs_with_metadata.csv', index=False)
print("Saved merged file as plogs_with_metadata.csv")

---- plogs.csv ----
(17476374, 12)
    log_id  student_id  assignment_id  problem_id  \
0  8931626      558286         818971     1423359   
1  8931626      558286         818971     1423360   
2  8931626      558286         818971     1423361   
3  8931626      558286         818971     1423362   
4  8931626      558286         818971     1423363   

                         start_time  time_on_task answer_before_tutoring  \
0  2020-11-01 01:04:22.133000-05:00       374.106                   True   
1  2020-11-01 01:10:37.108000-05:00        13.073                   True   
2  2020-11-01 01:10:51.303000-05:00        11.239                   True   
3  2020-11-01 01:11:03.950000-05:00        11.224                   True   
4  2020-11-01 01:11:15.952000-05:00        13.118                   True   

   fraction_of_hints_used  attempt_count  answer_given  problem_completed  \
0                     0.0              1         False               True   
1                     0.0          

In [11]:
import pandas as pd
import ast

def explode_skills(df):
    """
    Takes a DataFrame with a 'skills' column (list as string) and returns the DataFrame exploded on this column.
    """
    df = df.copy()
    df = df[df['skills'].notnull()]
    df['skills_list'] = df['skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else [])
    exploded = df.explode('skills_list')
    exploded = exploded.rename(columns={'skills_list': 'skill'})
    return exploded

def skill_summary(df):
    """
    Summary per skill: attempts, unique students, mean correctness.
    """
    ex = explode_skills(df)
    skill_grp = ex.groupby('skill').agg(
        n_attempts = pd.NamedAgg(column='problem_id', aggfunc='count'),
        n_students = pd.NamedAgg(column='student_id', aggfunc=lambda x: x.nunique()),
        mean_correct = pd.NamedAgg(column='correct', aggfunc='mean')
    ).reset_index()
    return skill_grp.sort_values('n_attempts', ascending=False)

def student_skill_matrix(df, min_attempts=5):
    """
    For each student and skill: attempts and mean correctness. Filters to at least min_attempts (default=5).
    """
    ex = explode_skills(df)
    grouped = ex.groupby(['student_id','skill']).agg(
        n_attempts = pd.NamedAgg(column='problem_id', aggfunc='count'),
        mean_correct = pd.NamedAgg(column='correct', aggfunc='mean')
    ).reset_index()
    return grouped[grouped['n_attempts'] >= min_attempts]

def problem_difficulty_table(df, min_attempts=20):
    """
    For each problem: attempts, mean correctness, skill, content_source, type. Filters to at least min_attempts.
    """
    tbl = df.groupby('problem_id').agg(
        n_attempts = pd.NamedAgg(column='student_id', aggfunc='count'),
        mean_correct = pd.NamedAgg(column='correct', aggfunc='mean'),
        skill = pd.NamedAgg(column='skills', aggfunc='first'),
        source = pd.NamedAgg(column='content_source', aggfunc='first'),
        problem_type = pd.NamedAgg(column='problem_type', aggfunc='first')
    ).reset_index()
    return tbl[tbl['n_attempts']>=min_attempts].sort_values('mean_correct')

def content_source_distribution(df):
    """
    Simple value count of attempts per content_source.
    """
    return df['content_source'].value_counts().reset_index(name='n_attempts').rename(columns={'index': 'content_source'})

In [3]:
merged = pd.read_csv('plogs_with_metadata.csv')

In [8]:
merged['correct'].value_counts(dropna=False)

correct
True     8449108
NaN      4732693
False    4294573
Name: count, dtype: int64

In [10]:
skill_dist = skill_summary(merged)
skill_dist.head(10)

,skill,n_attempts,n_students,mean_correct
258,7.RP.A.2a,386593,29472,0.624084
198,6.RP.A.3b,368498,19577,0.612861
262,7.RP.A.3,352508,21870,0.550095
176,6.NS.A.1,336854,14777,0.617123
26,2.NBT.B.5,314070,9030,0.759718
284,8.EE.C.7b,291914,16910,0.594142
197,6.RP.A.3a,287578,24364,0.612179
199,6.RP.A.3c-1,257786,13838,0.688688
260,7.RP.A.2c,228835,19741,0.587476
282,8.EE.B.6,208773,12618,0.578701


In [12]:
student_skills = student_skill_matrix(merged, min_attempts=5)
student_skills.head(10)

,student_id,skill,n_attempts,mean_correct
0,124,7.RP.A.3,10,0.1
7,2246,7.RP.A.2a,5,0.0
8,2246,8.EE.B.6,28,0.8
10,2246,8.EE.C.7b,6,0.0
12,2246,8.F.B.4-3,7,0.0
15,2246,8.F.B.4-8,6,0.4
16,2246,8.G.A.4,8,1.0
19,2259,8.EE.B.5,13,0.5
23,2259,8.F.B.4-8,8,0.571429
25,2271,8.EE.B.6,5,0.5


In [13]:
difficulty_tbl = problem_difficulty_table(merged, min_attempts=20)
difficulty_tbl.head(10)

,problem_id,n_attempts,mean_correct,skill,source,problem_type
79586,1581958,21,0.0,None,['Textbook Content'],Exact Match (ignore case)
76642,1543800,103,0.0,['6.EE.A.2a'],['Utah Math'],Numeric Expression
59025,1415783,40,0.0,None,['Engage New York'],Algebraic Expression
111014,1917270,24,0.0,None,['Undetermined'],Exact Match (ignore case)
37546,1099693,21,0.0,None,['Undetermined'],Algebraic Expression
86951,1663590,38,0.0,None,['Textbook Content'],Exact Match (ignore case)
85364,1632942,20,0.0,None,['Textbook Content'],Number
18217,448683,21,0.0,['8.EE.C.8b'],['Undetermined'],Exact Match (case sensitive)
66583,1472047,27,0.0,"['8.F.A.1', '8.F.B.4-8']",['Utah Math'],Ungraded Open Response
21991,660214,20,0.0,['6.RP.A.3b'],['Skill Builder'],Algebraic Expression


In [14]:
content_dist = content_source_distribution(merged)
content_dist

,content_source,n_attempts
0,['Engage New York'],10426991
1,"['Illustrative Mathematics', 'Open Up Resources']",3963013
2,['Illustrative Mathematics'],931348
3,['Undetermined'],742905
4,['Textbook Content'],601003
5,['Skill Builder'],350004
6,['Open Up Resources'],156705
7,['Certified Content'],131154
8,['Utah Math'],75064
9,['Eureka Math'],53956
